In [1]:
# !pip install syft

     |████████████████████████████████| 256kB 2.8MB/s 
     |████████████████████████████████| 204kB 42.3MB/s 
     |████████████████████████████████| 256kB 39.7MB/s 
     |████████████████████████████████| 1.4MB 40.9MB/s 
     |████████████████████████████████| 460kB 41.6MB/s 
     |████████████████████████████████| 389kB 41.4MB/s 
     |████████████████████████████████| 81kB 23.3MB/s 
     |████████████████████████████████| 266kB 40.3MB/s 
     |████████████████████████████████| 51kB 18.2MB/s 
     |████████████████████████████████| 122kB 42.8MB/s 
  Created wheel for zstd: filename=zstd-1.4.1.0-cp36-cp36m-linux_x86_64.whl size=1067085 sha256=d92af2f6da57f73161cc9aa98db938bdaa2469a18f914fd59032a73bff4d4abf
  Stored in directory: /root/.cache/pip/wheels/66/3f/ee/ac08c81af7c1b24a80c746df669ea3cb37542d27877d66ccf4
  Created wheel for pyyaml: filename=PyYAML-5.1.2-cp36-cp36m-linux_x86_64.whl size=44105 sha256=1d79db30d8c17e6e9788a33205807eef37648e860aad5528f484048a80b038cc
  Stored in di

# Section: Securing Federated Learning

- Lesson 1: Trusted Aggregator
- Lesson 2: Intro to Additive Secret Sharing
- Lesson 3: Intro to Fixed Precision Encoding
- Lesson 4: Secret Sharing + Fixed Precision in PySyft
- Final Project: Federated Learning wtih Encrypted Gradient Aggregation

# Lesson: Federated Learning with a Trusted Aggregator

In the last section, we learned how to train a model on a distributed dataset using Federated Learning. In particular, the last project aggregated gradients directly from one data owner to another. 

However, while in some cases it could be ideal to do this, what would be even better is to be able to choose a neutral third party to perform the aggregation.

As it turns out, we can use the same tools we used previously to accomplish this.

# Project: Federated Learning with a Trusted Aggregator

In [0]:
# try this project here!

In [2]:
import syft as sy
import torch as th
hook = sy.TorchHook(th)
from torch import nn, optim

W0806 12:46:17.427000 140265677387648 secure_random.py:26] Falling back to insecure randomness since the required custom op could not be found for the installed version of TensorFlow. Fix this by compiling custom ops. Missing file was '/usr/local/lib/python3.6/dist-packages/tf_encrypted/operations/secure_random/secure_random_module_tf_1.14.0.so'
W0806 12:46:17.444154 140265677387648 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/tf_encrypted/session.py:26: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.



In [0]:
# create a couple worker
bob = sy.VirtualWorker(hook, id='bob')
alice = sy.VirtualWorker(hook, id='alice')
secure_worker = sy.VirtualWorker(hook, id='secure_worker')

In [0]:
bob.add_workers([alice, secure_worker])
alice.add_workers([bob, secure_worker])
secure_worker.add_workers([alice, bob])

W0805 18:57:40.107830 4698277312 base.py:628] Worker alice already exists. Replacing old worker which could cause                     unexpected behavior
W0805 18:57:40.109066 4698277312 base.py:628] Worker secure_worker already exists. Replacing old worker which could cause                     unexpected behavior
W0805 18:57:40.109935 4698277312 base.py:628] Worker bob already exists. Replacing old worker which could cause                     unexpected behavior
W0805 18:57:40.110903 4698277312 base.py:628] Worker secure_worker already exists. Replacing old worker which could cause                     unexpected behavior
W0805 18:57:40.111799 4698277312 base.py:628] Worker alice already exists. Replacing old worker which could cause                     unexpected behavior
W0805 18:57:40.112837 4698277312 base.py:628] Worker bob already exists. Replacing old worker which could cause                     unexpected behavior


<VirtualWorker id:secure_worker #objects:0>

In [0]:
# A toy Dataset
data = th.tensor([[0,0], [0,1], [1,0], [1,1.]], requires_grad=True)
target = th.tensor([[0], [0], [1], [1.]], requires_grad=True)

In [0]:
# get pointers to training data on each worker by
# sending some training data to bob and alice
bobs_data = data[0:2].send(bob)
bobs_target = target[0:2].send(bob)

In [0]:
alices_data = data[2:].send(alice)
alices_target = target[2:].send(alice)

In [0]:
# Initialize A Toy Model
model = nn.Linear(2,1)

In [0]:
bobs_model = model.copy().send(bob)

In [0]:
alices_model = model.copy().send(alice)

In [0]:
bobs_opt = optim.SGD(params=bobs_model.parameters(), lr=0.1)
alices_opt = optim.SGD(params=alices_model.parameters(), lr=0.1)

In [0]:
bobs_opt.zero_grad()
bobs_pred = bobs_model(bobs_data)
bobs_loss = ((bobs_pred - bobs_target) ** 2).sum()
bobs_loss.backward()

(Wrapper)>[PointerTensor | me:29890115000 -> bob:39645682707]

In [0]:
bobs_opt.step()

In [0]:
bobs_loss = bobs_loss.get().data
bobs_loss

tensor(1.4823)

In [0]:
alices_opt.zero_grad()
alices_pred = alices_model(alices_data)
alices_loss = ((alices_pred - alices_target) ** 2).sum()
alices_loss.backward()

alices_opt.step()
alices_loss = alices_loss.get().data
alices_loss

tensor(0.1391)

In [0]:
for i in range(10):
    bobs_opt.zero_grad()
    bobs_pred = bobs_model(bobs_data)
    bobs_loss = ((bobs_pred - bobs_target) ** 2).sum()
    bobs_loss.backward()
    
    bobs_opt.step()
    bobs_loss = bobs_loss.get().data
    bobs_loss
    
    alices_opt.zero_grad()
    alices_pred = alices_model(alices_data)
    alices_loss = ((alices_pred - alices_target) ** 2).sum()
    alices_loss.backward()

    alices_opt.step()
    alices_loss = alices_loss.get().data
    alices_loss

In [0]:
bobs_model.move(secure_worker)
alices_model.move(secure_worker)

In [0]:
secure_worker._objects

{76558495992: Parameter containing:
 tensor([[0.1495, 0.0542]], requires_grad=True),
 9089294862: Parameter containing:
 tensor([-0.0332], requires_grad=True),
 18431426710: Parameter containing:
 tensor([[0.2421, 0.1908]], requires_grad=True),
 22866841631: Parameter containing:
 tensor([0.6507], requires_grad=True)}

In [0]:
alices_model

Linear(in_features=2, out_features=1, bias=True)

In [0]:
alices_model.weight

Parameter containing:
Parameter>[PointerTensor | me:18431426710 -> secure_worker:18431426710]

In [0]:
alices_model.weight.data

(Wrapper)>[PointerTensor | me:18431426710 -> secure_worker:18431426710]::data

In [0]:
model.weight.data = (((alices_model.weight.data + bobs_model.weight.data) / 2).get())

In [0]:
model.weight.data

tensor([[0.1958, 0.1225]])

In [0]:
model.bias.data = (((alices_model.bias.data + bobs_model.bias.data) / 2).get())

In [0]:
model.bias.data

tensor([0.3088])

In [0]:
for round_iter in range(10):
    
    bobs_model = model.copy().send(bob)
    alices_model = model.copy().send(alice)
    
    bobs_opt = optim.SGD(params=bobs_model.parameters(), lr=0.1)
    alices_opt = optim.SGD(params=alices_model.parameters(), lr=0.1)
    
    for i in range(10):
        bobs_opt.zero_grad()
        bobs_pred = bobs_model(bobs_data)
        bobs_loss = ((bobs_pred - bobs_target) ** 2).sum()
        bobs_loss.backward()

        bobs_opt.step()
        bobs_loss = bobs_loss.get().data
        bobs_loss

        alices_opt.zero_grad()
        alices_pred = alices_model(alices_data)
        alices_loss = ((alices_pred - alices_target) ** 2).sum()
        alices_loss.backward()

        alices_opt.step()
        alices_loss = alices_loss.get().data
        alices_loss
        
    bobs_model.move(secure_worker)
    alices_model.move(secure_worker)
    
    model.weight.data = (((alices_model.weight.data + bobs_model.weight.data) / 2).get())
    model.bias.data = (((alices_model.bias.data + bobs_model.bias.data) / 2).get())
    
    secure_worker.clear_objects()
    
    print('Bob:' + str(bobs_loss) + 'Alice:' + str(alices_loss))

Bob:tensor(0.0003)Alice:tensor(0.0050)
Bob:tensor(0.0012)Alice:tensor(0.0017)
Bob:tensor(0.0015)Alice:tensor(0.0008)
Bob:tensor(0.0014)Alice:tensor(0.0004)
Bob:tensor(0.0011)Alice:tensor(0.0003)
Bob:tensor(0.0009)Alice:tensor(0.0002)
Bob:tensor(0.0007)Alice:tensor(0.0001)
Bob:tensor(0.0005)Alice:tensor(9.1678e-05)
Bob:tensor(0.0004)Alice:tensor(6.7979e-05)
Bob:tensor(0.0003)Alice:tensor(5.0766e-05)


# Lesson: Intro to Additive Secret Sharing

While being able to have a trusted third party to perform the aggregation is certainly nice, in an ideal setting we wouldn't have to trust anyone at all. This is where Cryptography can provide an interesting alterantive. 

Specifically, we're going to be looking at <font color=red>a simple protocol for Secure Multi-Party Computation called Additive Secret Sharing.</font> <font color=blue>This protocol will allow multiple parties (of size 3 or more) to aggregate their gradients without the use of a trusted 3rd party to perform the aggregation.</font><font color=red> In other words, we can add 3 numbers together from 3 different people without anyone ever learning the inputs of any other actors.</font>

Let's start by considering the number 5, which we'll put into a varible x

In [0]:
x = 5

Let's say we wanted to SHARE the ownership of this number between two people, Alice and Bob. We could split this number into two shares, 2, and 3, and give one to Alice and one to Bob

In [4]:
bob_x_share = 2
alice_x_share = 3

decrypted_x = bob_x_share + alice_x_share
decrypted_x

5

Note that neither Bob nor Alice know the value of x. They only know the value of their own SHARE of x. Thus, the true value of X is hidden (i.e., encrypted). 

The truly amazing thing, however, is that Alice and Bob can still compute using this value! They can perform arithmetic over the hidden value! Let's say Bob and Alice wanted to multiply this value by 2! If each of them multiplied their respective share by 2, then the hidden number between them is also multiplied! Check it out!

In [5]:
bob_x_share = 2 * 2
alice_x_share = 3 * 2

decrypted_x = bob_x_share + alice_x_share
decrypted_x

10

This even works for addition between two shared values!!

In [7]:
# encrypted "5"
bob_x_share = 2
alice_x_share = 3

# encrypted "7"
bob_y_share = 5
alice_y_share = 2

# encrypted 5 + 7
bob_z_share = bob_x_share + bob_y_share
alice_z_share = alice_x_share + alice_y_share

decrypted_z = bob_z_share + alice_z_share
decrypted_z

12

As you can see, we just added two numbers together while they were still encrypted!!!

One small tweak - notice that since all our numbers are positive, it's possible for each share to reveal a little bit of information about the hidden value, namely, it's always greater than the share. Thus, if Bob has a share "3" then he knows that the encrypted value is at least 3.

This would be quite bad, but can be solved through a simple fix. <font color=red>Decryption happens by summing all the shares together MODULUS some constant. I.e.</font>

In [8]:
x = 5

Q = 23740629843760239486723

bob_x_share = 23552870267 # <- a random number
alice_x_share = Q - bob_x_share + x
alice_x_share

23740629843736686616461

In [9]:
(bob_x_share + alice_x_share) % Q

5

So now, as you can see, both shares are wildly larger than the number being shared, meaning that individual shares no longer leak this inforation. However, all the properties we discussed earlier still hold! (addition, encryption, decryption, etc.)

# Project: Build Methods for Encrypt, Decrypt, and Add 

In this project, you must take the lessons we learned in the last section and write general methods for encrypt, decrypt, and add. Store shares for a variable in a tuple like so.

In [0]:
x_share = (2,5,7)

Even though normally those shares would be distributed amongst several workers, you can store them in ordered tuples like this for now :)

In [0]:
# try this project here!

In [0]:
import random

In [0]:
Q = 231515164481911616167

In [0]:
x = 5

In [0]:
# def encrypt(x, n_shares=3):
n_shares=3
shares = list()

for i in range(n_shares - 1):
    shares.append(random.randint(0,Q))

In [0]:
final_share = Q - (sum(shares) % Q) + x

In [0]:
shares.append(final_share)

In [22]:
sum(shares) % Q

5

In [0]:
def encrypt(x, n_shares=3):
    
    shares = list()

    for i in range(n_shares - 1):
        shares.append(random.randint(0,Q))
    
    final_share = Q - (sum(shares) % Q) + x
    
    shares.append(final_share)
    
    return tuple(shares)

In [24]:
encrypt(5, n_shares=10)

(70406955963263575538,
 119678468524038469409,
 37102692774022703909,
 129853415938876106875,
 22544037772758356658,
 194000789882642826852,
 194188907992462387774,
 186099931477988509871,
 78369464445317418630,
 125331157638187725324)

In [0]:
def decrypt(shares):
    return sum(shares) % Q

In [0]:
def add(a, b):
    
    c = list()
    
    assert(len(a) == len(b))
    
    for i in range(len(a)):
        c.append((a[i] + b[i]) % Q)
        
    return tuple(c)

In [27]:
decrypt(add(encrypt(5), encrypt(10)))

15

In [28]:
x = encrypt(5)
x

(205363750793521099300, 90201576011055395661, 167465002159246737378)

In [29]:
y = encrypt(10)
y

(179195696590513060671, 209963358381002512399, 73871273992307659274)

In [30]:
z = add(x, y)
z

(153044282902122543804, 68649769910146291893, 9821111669642780485)

In [31]:
decrypt(z)

15

# Lesson: Intro to Fixed Precision Encoding

As you may remember, our goal is to aggregate gradients using this new Secret Sharing technique. However, the protocol we've just explored in the last section uses positive integers. However, our neural network weights are NOT integers. Instead, our weights are decimals (floating point numbers).

Not a huge deal! We just need to use a fixed precision encoding, which lets us do computation over decimal numbers using integers!

In [0]:
BASE = 10
PRECISION = 4
Q = 231515164481911616167

In [0]:
def encode(x):
    return int((x * (BASE ** PRECISION)) % Q)

def decode(x):
    return (x if x <= Q/2 else x - Q) / BASE**PRECISION

In [0]:
encode(0.5)

5000

In [0]:
decode(5000 + 5000)

1.0

In [0]:
x = encrypt(encode(5.5))
y = encrypt(encode(2.3))
z = add(x,y)
decode(decrypt(z))

NameError: name 'encrypt' is not defined

# Lesson: Secret Sharing + Fixed Precision in PySyft

While writing things from scratch is certainly educational, PySyft makes a great deal of this much easier for us through its abstractions.

In [0]:
bob = bob.clear_objects()
alice = alice.clear_objects()
secure_worker = secure_worker.clear_objects()

In [0]:
x = th.tensor([1,2,3,4,5])

### Secret Sharing Using PySyft

We can share using the simple .share() method!

In [0]:
x = x.share(bob, alice, secure_worker)
x

(Wrapper)>[AdditiveSharingTensor]
	-> (Wrapper)>[PointerTensor | me:49829662577 -> bob:86995293157]
	-> (Wrapper)>[PointerTensor | me:63476300362 -> alice:75532519202]
	-> (Wrapper)>[PointerTensor | me:52747809027 -> secure_worker:64987187112]
	*crypto provider: me*

In [0]:
bob._objects

{86995293157: tensor([3690660184154563446,  637462768570026377,  348577348449200103,
         1847927098397550801, 2280779949752776534])}

and as you can see, Bob now has one of the shares of x! Furthermore, we can still call addition in this state, and PySyft will automatically perform the remote execution for us!

In [0]:
y = x + x

In [0]:
bob._objects

{86995293157: tensor([3690660184154563446,  637462768570026377,  348577348449200103,
         1847927098397550801, 2280779949752776534]),
 67328084747: tensor([7381320368309126892, 1274925537140052754,  697154696898400206,
         3695854196795101602, 4561559899505553068])}

In [0]:
y

(Wrapper)>[AdditiveSharingTensor]
	-> (Wrapper)>[PointerTensor | me:62964429758 -> bob:67328084747]
	-> (Wrapper)>[PointerTensor | me:47364154149 -> alice:64819179181]
	-> (Wrapper)>[PointerTensor | me:26595940338 -> secure_worker:19348821163]
	*crypto provider: me*

In [0]:
y.get()

tensor([ 2,  4,  6,  8, 10])

In [0]:
bob._objects

{86995293157: tensor([3690660184154563446,  637462768570026377,  348577348449200103,
         1847927098397550801, 2280779949752776534])}

### Fixed Precision using PySyft

We can also convert a tensor to fixed precision using .fix_precision()

In [0]:
x = th.tensor([0.1,0.2,0.3])

In [0]:
x

tensor([0.1000, 0.2000, 0.3000])

In [0]:
x = x.fix_prec()

In [0]:
x

(Wrapper)>FixedPrecisionTensor>tensor([100, 200, 300])

In [0]:
x = x.float_prec()
x

tensor([0.1000, 0.2000, 0.3000])

In [0]:
x = th.tensor([0.1,0.2,0.3])
x

tensor([0.1000, 0.2000, 0.3000])

In [0]:
x = x.fix_prec()
x

(Wrapper)>FixedPrecisionTensor>tensor([100, 200, 300])

In [0]:
type(x)

syft.frameworks.torch.tensors.interpreters.native.Tensor

In [0]:
type(x.child)

syft.frameworks.torch.tensors.interpreters.precision.FixedPrecisionTensor

In [0]:
type(x.child.child)

syft.frameworks.torch.tensors.interpreters.native.Tensor

In [0]:
x.child.child

tensor([100, 200, 300])

In [0]:
y = x + x
y

(Wrapper)>FixedPrecisionTensor>tensor([200, 400, 600])

In [0]:
y = y.float_prec()
y

tensor([0.2000, 0.4000, 0.6000])

### Shared Fixed Precision

And of course, we can combine the two!

In [0]:
x = th.tensor([0.1, 0.2, 0.3])

In [0]:
x = x.fix_prec().share(bob, alice, secure_worker)
x

(Wrapper)>FixedPrecisionTensor>(Wrapper)>[AdditiveSharingTensor]
	-> (Wrapper)>[PointerTensor | me:74014187610 -> bob:15444838458]
	-> (Wrapper)>[PointerTensor | me:65252605033 -> alice:825053878]
	-> (Wrapper)>[PointerTensor | me:44256523821 -> secure_worker:90945091752]
	*crypto provider: me*

In [0]:
y = x + x
y

(Wrapper)>FixedPrecisionTensor>(Wrapper)>[AdditiveSharingTensor]
	-> (Wrapper)>[PointerTensor | me:94384448434 -> bob:68707277136]
	-> (Wrapper)>[PointerTensor | me:91505907959 -> alice:62655741142]
	-> (Wrapper)>[PointerTensor | me:82998294336 -> secure_worker:20565041377]
	*crypto provider: me*

In [0]:
y.get().float_prec()

tensor([0.2000, 0.4000, 0.6000])

Make sure to make the point that people can see the model averages in the clear.

# Final Project: Federated Learning with Encrypted Gradient Aggregation